# ETL Gerenciador de Ambientes Laboratoriais (GAL) data for ITpS analyses

In [67]:
## Libs
import pandas as pd
import hashlib
from epiweeks import Week, Year


In [70]:
pwd

'/Users/bragatte/Documents/GitHub/arbo'

In [72]:
## read files combined_arbo.xlsx and gal_am_12_12_23.txt
### ITpS
combined_arbo = pd.read_excel('EDA/data/combined/combined_arbo.xlsx')
### GAL
gal_am_12_12_23 = pd.read_csv('EDA/data/cglab_fake/gal_am_12_12_23', sep=';')

In [74]:
## Check structure of files
### ITpS
# print(combined_arbo.head())
## list of columns
print(combined_arbo.columns)
## list of unique values in column
print(combined_arbo['test_kit'].unique())

Index(['lab_id', 'sample_id', 'test_id', 'test_kit', 'gender', 'age',
       'location', 'date_testing', 'state', 'patient_id', 'file_name',
       'denv_test_result', 'zikv_test_result', 'chikv_test_result',
       'yfv_test_result', 'mayv_test_result', 'orov_test_result',
       'wnv_test_result', 'qty_original_lines', 'created_at', 'updated_at',
       'age_group', 'epiweek', 'month', 'country', 'region', 'macroregion',
       'macroregion_code', 'state_code', 'state_ibge_code',
       'location_ibge_code', 'lat', 'long'],
      dtype='object')
['igm_serum' 'igg_serum' 'ns1_antigen' 'arbo_pcr_3']


In [75]:
## Check structure of files
### GAL
# print(gal_am_12_12_23.head())
## list of columns
print(gal_am_12_12_23.columns)
## list of unique values in column
print(gal_am_12_12_23['status_exame'].unique())
print(gal_am_12_12_23['resultado'].unique())

## list of unique values in column resultado == Reagente
print(gal_am_12_12_23[gal_am_12_12_23['resultado'] == 'Reagente ']['status_exame'].unique())

Index(['Unnamed: 0', 'requisicao', 'setor', 'bancada', 'mun_residencia',
       'uf_residencia', 'requisitante', 'mun_requisitante', 'exame', 'metodo',
       'material', 'cod_amostra', 'amostra', 'restricao',
       'laboratorio_cadastro', 'dt_cadastro', 'dt_recebimento', 'dt_liberacao',
       'tempo_liberacao', 'laboratorio_executor', 'status_exame', 'resultado'],
      dtype='object')
['Resultado Liberado' 'Exame não-realizado' 'Aguardando Triagem'
 'Exame em Análise']
['Não Reagente ' ' ' 'Reagente ' 'Indeterminado ']
['Resultado Liberado']


In [78]:
## check unique values for status_exame
print(gal_am_12_12_23['status_exame'].unique())

['Resultado Liberado']


In [77]:
## filter status_exame == 'Resultado Liberado' and update gal_am_12_12_23
gal_am_12_12_23 = gal_am_12_12_23[gal_am_12_12_23['status_exame'] == 'Resultado Liberado']

## MAP columns and variables

In [79]:
## Create a new DataFrame with the same columns as combined_arbo
## Initially, all columns are empty
combined_structure_columns = [
    'lab_id', 'sample_id', 'test_id', 'test_kit', 'gender', 'age',
    'location', 'date_testing', 'state', 'patient_id', 'file_name',
    'denv_test_result', 'zikv_test_result', 'chikv_test_result',
    'yfv_test_result', 'mayv_test_result', 'orov_test_result',
    'wnv_test_result', 'qty_original_lines', 'created_at', 'updated_at',
    'age_group', 'epiweek', 'month', 'country', 'region', 'macroregion',
    'macroregion_code', 'state_code', 'state_ibge_code',
    'location_ibge_code', 'lat', 'long'
]
denv_gal = pd.DataFrame(columns=combined_structure_columns)

In [80]:
## Populate columns according to instructions
denv_gal['lab_id'] = 'CGLAB'  # Fixed value
denv_gal['test_id'] = gal_am_12_12_23['cod_amostra']
denv_gal['test_kit'] = gal_am_12_12_23['exame'].replace({'Dengue, Detecção de Antígeno NS1': 'ns1_antigen'})
denv_gal['location'] = gal_am_12_12_23['mun_residencia']
denv_gal['state_code'] = gal_am_12_12_23['uf_residencia']
denv_gal['date_testing'] = pd.to_datetime(gal_am_12_12_23['dt_cadastro'], format='%d/%m/%Y %H:%M:%S')
denv_gal['denv_test_result'] = gal_am_12_12_23['resultado'].apply(
    lambda x: 'Pos' if x.strip() == 'Reagente' else
    ('Neg' if x.strip() == 'Não Reagente' else 'NT')
)

In [81]:
## Populate columns according to instructions
def get_epiweek_end_date(row):
    """
    Convert a date to the end date of its epidemiological week.
    Only updates if epiweek is empty.

    :param row: DataFrame row.
    :return: End date of the epidemiological week or existing epiweek value.
    """
    try:
        ## Check if epiweek is already set
        if pd.isna(row['epiweek']) or row['epiweek'] == '':
            date = pd.to_datetime(row['date_testing'])
            epiweek = Week.fromdate(date, system="cdc")
            return str(epiweek.enddate())
        else:
            return row['epiweek']
    except Exception as e:
        ## Log the exception or return a default value if the date is invalid
        return ''

## Check if epiweek column exists and apply the function accordingly
if 'epiweek' in denv_gal.columns:
    denv_gal['epiweek'] = denv_gal.apply(get_epiweek_end_date, axis=1)
else:
    denv_gal['epiweek'] = denv_gal['date_testing'].apply(lambda x: get_epiweek_end_date({'date_testing': x, 'epiweek': ''}))


## Generate hash for sample_id based on specific columns of gal_am_12_12_23
def generate_hash(row):
    hash_input = str(row['requisicao']) + str(row['cod_amostra'])
    return hashlib.sha1(hash_input.encode('utf-8')).hexdigest()

## Applying the hash generation function for the sample_id column
denv_gal['sample_id'] = gal_am_12_12_23.apply(generate_hash, axis=1)


In [82]:
## Save the transformed DataFrame to a TSV file
tsv_file_path = 'EDA/results/combined_arbo_gal.tsv'
denv_gal.to_csv(tsv_file_path, sep='\t', index=False)

## Pos Neg

In [84]:
## Group data by epiweek and count the positive and negative cases
denv_posneg = denv_gal.groupby('epiweek')['denv_test_result'].value_counts().unstack().fillna(0)
denv_posneg['total'] = denv_posneg['Pos'] + denv_posneg['Neg']

## Rate

In [85]:
## Calculate the rate of positive cases
denv_posneg['Pos_Rate'] = denv_posneg['Pos'] / denv_posneg['total']

## Reset the index to turn epiweek from an index into a column
denv_posneg.reset_index(inplace=True)

## Select only the columns of interest (epiweek and Pos_Rate)
denv_rate = denv_posneg[['epiweek', 'Pos_Rate']]

## Display the new DataFrame
print(denv_rate)

denv_test_result     epiweek  Pos_Rate
0                 2023-01-07  0.096154
1                 2023-01-14  0.157895
2                 2023-01-21  0.173077
3                 2023-01-28  0.118421
4                 2023-02-04  0.202899
5                 2023-02-11  0.120000
6                 2023-02-18  0.076923
7                 2023-02-25  0.107692
8                 2023-03-04  0.194444
9                 2023-03-11  0.052632
10                2023-03-18  0.176991
11                2023-03-25  0.180851
12                2023-04-01  0.115942
13                2023-04-08  0.172840
14                2023-04-15  0.227848
15                2023-04-22  0.254237
16                2023-04-29  0.215827
17                2023-05-06  0.166667
18                2023-05-13  0.290780
19                2023-05-20  0.192593
20                2023-05-27  0.198198
21                2023-06-03  0.155738
22                2023-06-10  0.155844
23                2023-06-17  0.126582
24                2023-06

## Plot

In [86]:
import plotly.express as px

## Plot the data using Plotly
fig = px.line(denv_rate, x='epiweek', y='Pos_Rate', title='Dengue Positive Rate by Epidemiological Week', labels={'Pos_Rate': 'Positive Rate', 'epiweek': 'Epidemiological Week'})
fig.show()

In [88]:
## fazer gráfico de barras mostrando os resultados positivos e negativos usando plotly
fig = px.bar(denv_posneg, x='epiweek', y=['Pos', 'Neg'], title='Dengue Positive and Negative Cases by Epidemiological Week', labels={'value': 'Cases', 'variable': 'Result', 'epiweek': 'Epidemiological Week'})
fig.show()